# Imports

In [10]:
# Data management
import numpy as np
import pandas as pd

# Data fetching
import yfinance as yf

# Backtesting

# ML

# Utils
from datetime import datetime
from datetime import timedelta

# Data fetch

In [12]:
def get_stonk_data(stonk_list, date_start, date_end=datetime.now(), interval='1d', source='yfinance', proxy=False):
    '''
    Returns historical price data for the selected financial instrument.

        Parameters:
            Required:
                stonk_list (string, list):
                    List of stonk identifiers as strings
                date_start (datetime):
                    Start date for stonk data
            Optional:
                date_end (datetime):
                    End date for stonk data
                interval (string)
                    Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                source (string):
                    Where to source data from. Valid sources: yfinance
                proxy (boolean):
                    Whether to use a proxy connection to avoid API limits/blocks
                
        Returns:
            stonk_data (Pandas Dataframe): Pandas Dataframe containing requested data
    '''
        
    if source == 'yfinance':
        stonks = yf.download(stonk_list, start=date_start, end=date_end, interval=interval, group_by='column', auto_adjust=True, threads=True, rounding=True)['Adj Close']
    else:
        raise ValueError('Unsupported data source type')
    
    

In [27]:
stonks = yf.download(["msft", "tsla"], period='1y', interval='1d', group_by='column')

[*********************100%***********************]  2 of 2 completed


In [28]:
stonks['Adj Close']

,MSFT,TSLA
Date,,
2021-02-05,240.186691,852.229980
2021-02-08,240.454437,863.419983
2021-02-09,241.743622,849.460022
2021-02-10,240.801529,804.820007
2021-02-11,242.457657,811.659973
...,...,...
2022-01-31,310.980011,936.719971
2022-02-01,308.760010,931.250000
2022-02-02,313.459991,905.659973


In [11]:
help(yf.download)

Help on function download in module yfinance.multi:

download(tickers, start=None, end=None, actions=False, threads=True, group_by='column', auto_adjust=False, back_adjust=False, progress=True, period='max', show_errors=True, interval='1d', prepost=False, proxy=None, rounding=False, timeout=None, **kwargs)
    Download yahoo tickers
    :Parameters:
        tickers : str, list
            List of tickers to download
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime.
            Default is 1900-01-01
        end: str
            Download end date string (YYYY-MM-DD) or _datetime.
            Default is now
        group_by : str
            Group by 'ticker' o

In [8]:
help(yf.Ticker)

Help on class Ticker in module yfinance.ticker:

class Ticker(yfinance.base.TickerBase)
 |  Ticker(ticker, session=None)
 |  
 |  Method resolution order:
 |      Ticker
 |      yfinance.base.TickerBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  option_chain(self, date=None, proxy=None, tz=None)
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  actions
 |  
 |  analysis
 |  
 |  balance_sheet
 |  
 |  balancesheet
 |  
 |  calendar
 |  
 |  cashflow
 |  
 |  dividends
 |  
 |  earnings
 |  
 |  financials
 |  
 |  info
 |  
 |  institutional_holders
 |  
 |  isin
 |  
 |  major_holders
 |  
 |  mutualfund_holders
 |  
 |  news
 |  
 |  options
 |  
 |  quarterly_balance_sheet
 |  
 |  quarterly_balancesheet
 |  
 |  quarterly_cashflow
 |  
 |  quarterly_earnings
 |  
 |  quarterly_financials
 |  
 |  recommendations
 |  
 |  shares
 |  
 |

# Backtesting